# Setup

In [1]:
#=
import Pkg;
Pkg.add("StaticArrays");
Pkg.status()
=#

In [2]:
using StaticArrays;

In [3]:
struct Square{T}
    x::Float64
    y::Float64
    w::Float64
    dielectric::T
end

struct GriddedArray{T}
    xlims::Tuple{Float64, Float64}
    ylims::Tuple{Float64, Float64}
    pixels::Matrix{T}
end

const TensorGriddedArray = GriddedArray{SMatrix{2, 2, Float64}}

TensorGriddedArray (alias for GriddedArray{SArray{Tuple{2, 2}, Float64, 2, L} where L})

# Question 1 

---

# Work
Consider the solution to this problem for a single element of matrix A called $A_{ij}$

`raster_area!` should find:
> the fraction of area that pixel intersects with s

A diagram of this situation shows that the same problem can be rephrased as **the fraction of area inside s relative to the total pixel area**

*Note:  There might be an approach that uses the "centers bounding box" relationship to Square that calculates the compliment of overlapping area*

Then $A_{i,j} = \frac{Intersectional Area}{Total Pixel Area}$

## Total Pixel Area
Clearly, $A_{pixel, total} = w_{pixel} \times h_{pixel}$

Known information is:
- xlims
- ylims
- pixels (matrix)

From the example, pixel dimensions can be calculated this way:

$w_{pixels} = 2 \times$ [(length of xlims) / (num columns in matrix)]

Similar formula for the pixel height

(Assuming) each pixel is the same size, pixel width, pixel height, and total pixel area only need to be calculated once

## Intersectional Area

When a pixel and square overlap, the resulting shape is a rectangle with some width and height

So, the intersectional area is $A_{p\cup s} = w_{p\cup s} \times h_{p\cup s}$

Using this shorthand to describe the corners of a rectangle:
- URC => Upper Right Corner
- LLC => Lower Left Corner
- ...

(Assuming all square and pixel edges are orthogonal to the coordinate axes (i.e. x-y plane)) this formula computes the dimensions of the new rectangle

Let w' = min(s.LRC, p.LRC) - max(s.LLC, p.LLC) 

Let h' = min(s.URC, p.URC) - max(s.LRC, p.LRC) 

Then $A_{p\cup s} = $ max(0, w') $\times$ max(0, h')

*Note:  If the orthogonal assumption is not always true, I think I would ammend the solution by first applying a rotation matrix, solving the problem, then rotating back*

### Square Corners and Pixel Corners

Known information for **Square** is 
- center x coord
- center y coord
- width

Then corners are given by combinations of known information:

$(x \pm \frac{w}{2}, y \pm \frac{w}{2})$

---

Interfacing with pixels is the same as accessing an element of the matrix `GriddedArray.pixels`

~~A datatype that represents pixels as rectangles would let w' and h' formulas to be written as above
Having a custom datatype would also allow for multiple dispatch, packaging pixel dimension computations into a one line constructor~~

```
BoxedPixel(GriddedArray)
    width
    height
    total area
end

BoxedPixel(GriddedArray, MatrixElement)
    index
    x
    y
    width
    height
    total area?
end 
```

~~Now define LRC, URC, LLC, ULC for Square and BoxedPixel~~

~~Formulas above give intersectional area~~

---

Instead of having to deal with uninitialized fields in a struct constructor, dispatch the corner finding functions for 2 types
- Square
- Matrix

**Edge locations convey the same information**

In [4]:
function pixel_width(A::GriddedArray)
    ncols = size(A.pixels, 2)
    (A.xlims[2] - A.xlims[1]) / (ncols - 1)

end

function pixel_height(A::GriddedArray)
    nrows = size(A.pixels, 1)
    (A.ylims[2] - A.ylims[1]) / (nrows-1)
end

pixel_height (generic function with 1 method)

In [5]:
function pixel_center(A::GriddedArray, index::Int64) 

    nrows = size(A.pixels, 1)
    ncols = size(A.pixels, 2)
    
    Δx = pixel_width(A)
    Δy = pixel_height(A)
    
    xcenter = A.xlims[1] + Δx * ((index - 1.0) % ncols) 
    ycenter = A.ylims[1] + Δy * ((index - 1.0) ÷ ncols)
    
    (xcenter, ycenter)
end

pixel_center (generic function with 1 method)

In [6]:
total_pixel_area(A::GriddedArray) = pixel_width(A) * pixel_height(A)

total_pixel_area (generic function with 1 method)

**Represent Squares and Pixels as rectangles**

In [7]:
# Square
right_edge(s::Square) = s.x + s.w/2
left_edge(s::Square) = s.x - s.w/2
top_edge(s::Square) = s.y + s.w/2
bot_edge(s::Square) = s.y - s.w/2

# Matrix
function right_edge(A::GriddedArray, index::Int64)
    # Find x coord pixel center
    xcenter, _ = pixel_center(A, index)
    # Find pixel width
    w = pixel_width(A)
    # Compute edge loc
    xcenter + w/2
end

function left_edge(A::GriddedArray, index::Int64)
    # Find x coord pixel center
    xcenter, _ = pixel_center(A, index)
    # Find pixel width
    w = pixel_width(A)
    # Compute edge loc
    xcenter - w/2
end

function top_edge(A::GriddedArray, index::Int64)
    # Find y coord pixel center
    _, ycenter = pixel_center(A, index)
    # Find pixel height
    h = pixel_height(A)
    # Compute edge loc
    ycenter + h/2
end

function bot_edge(A::GriddedArray, index::Int64)
    # Find y coord pixel center
    _, ycenter = pixel_center(A, index)
    # Find pixel height
    h = pixel_height(A)
    # Compute edge loc
    ycenter - h/2
end

bot_edge (generic function with 2 methods)

**Use the rectangle representation to compute the area of overlap**

In [8]:
# Overlapping rectangle

function w_prime(A::GriddedArray, index::Int64, s::Square)
   min(right_edge(s), right_edge(A, index)) - max(left_edge(s), left_edge(A, index)) 
end

function h_prime(A::GriddedArray, index::Int64, s::Square)
   min(top_edge(s), top_edge(A, index)) - max(bot_edge(s), bot_edge(A, index))
end

function overlapping_area(A::GriddedArray, index::Int64, s::Square)
    max(0, w_prime(A, index, s)) * max(0, h_prime(A, index, s))
end

overlapping_area (generic function with 1 method)

**Question 1 - `raster_area!` function

In [9]:
function raster_area!(A::GriddedArray, s::Square)
    # Calculate the total area of a single pixel
    pixel_area = total_pixel_area(A)

    # Iterate over the pixels in A
    for index in eachindex(A.pixels)
        # Calculate the overlapping area
        overlap_area = overlapping_area(A, index, s)
        # Calculate the target
        # Assign the value of the target to the current element in A
        A.pixels[index] = (overlap_area / pixel_area)
    end
    A
end

raster_area! (generic function with 1 method)

### Raster Area Examples

Several examples to check `raster_area!`

#### Square completely overlaps all the pixels

In [10]:
# GriddedArray specified in the set up example
A = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(1.0, 2, 2))

# Square with area that totally overlaps all pixels
# For Question 1, the dielectric value is arbitrary
very_large_square = Square(0.0, 0.0, 9999.0, 1.0)

# Expect that pixels matrix is `ones`
raster_area!(A, very_large_square)

GriddedArray{Float64}((0.0, 1.0), (0.0, 2.0), [1.0 1.0; 1.0 1.0])

#### Square does not overlap any pixel

In [11]:
# GriddedArray specified in the set up example
A = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(1.0, 2, 2))

# Square with small area that is very far away from all pixels
# For Question 1, the dielectric value is arbitrary
far_away_square = Square(-1000.0, -1000.0, 1.0, 1.0)

# Expect that pixels matrix is `zeros`
raster_area!(A, far_away_square)

GriddedArray{Float64}((0.0, 1.0), (0.0, 2.0), [0.0 0.0; 0.0 0.0])

#### Constructed Example 

Pen/paper example worked out such that pixels matrix is `[[0.25, 0.5], [0.5, 1.0]]`

In [12]:
# GriddedArray specified in the set up example
A = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(1.0, 2, 2))

#= 
Constructed example of a Square that overlaps the pixels giving

[[0.25, 0.5], [0.5, 1.0]]

For Question 1, the dielectric value is arbitrary
=#
constructed_square = Square(1.5, 1.5, 3.0, 1.0)

raster_area!(A, constructed_square)

GriddedArray{Float64}((0.0, 1.0), (0.0, 2.0), [0.25 0.5; 0.5 1.0])

#### Bigger Pixel Matrix

Example where the underlying pixels matrix is 5x5 instead of 2x2

In [13]:
# GriddedArray that is bigger than the example GriddedArray
A_55 = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(1.0, 5, 5))

#=
Constructed example where the pixel matrix is 5x5
=#
constructed_square_55 = Square(0.5, 1.0, 0.75, 1.0)

raster_area!(A_55, constructed_square_55)

# View pixels matrix by itself
A_55.pixels

5×5 Matrix{Float64}:
 0.0  0.0   0.0  0.0   0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.0   0.0  0.0   0.0

For each pixel, its center is located on or within the *centers bounding box* (a rectangle defined by `xlims` and `ylims`)

The dimensions of each pixel are calculated by evenly dividing the width and height of the centers bounding box.

Drawing a few diagram examples shows that each dimension of the centers bounding box is divided into $d-1$ even sections, where $d$ is either the number of rows or the number of columns in the pixels matrix.

Using this rule, the pixels from the above example should have width $\frac{1}{5-1}$ and height $\frac{2}{5-1}$. The resulting area is $\frac{1}{8}$ or 0.125

In [14]:
total_pixel_area(A_55)

0.125

In [15]:
sum(A_55.pixels * total_pixel_area(A_55)) == (constructed_square_55.w)^2

true

It will be useful to construct the raster area matrix, as in Question 1, without modifying the underlying matrix.

In [16]:
function raster_area(A::GriddedArray, s::Square)
    
    # Create a copy of the unmodified GriddedArray
    # FIXME:  Potentially expensive operation
    A′ = deepcopy(A)
    
    # Calculate the total area of a single pixel
    pixel_area = total_pixel_area(A′)

    # Iterate over the pixels in A
    for index in eachindex(A′.pixels)
        # Calculate the overlapping area
        overlap_area = overlapping_area(A′, index, s)
        # Calculate the target
        # Assign the value of the target to the current element in A
        A′.pixels[index] = (overlap_area / pixel_area)
    end
    A′
end

raster_area (generic function with 1 method)

Check that `raster_area` behaves as intended

In [17]:
original = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(1.0, 5, 5))
original.pixels

5×5 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

In [18]:
result = raster_area(original, constructed_square_55)
result.pixels

5×5 Matrix{Float64}:
 0.0  0.0   0.0  0.0   0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.25  1.0  0.25  0.0
 0.0  0.0   0.0  0.0   0.0

In [19]:
original.pixels

5×5 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

# Question 2

---

# Work

For each pixel, calculate:

$\langle\varepsilon\rangle = (\kappa_{pixel})(1-A_{overlap}) + (\kappa_{square})(A_{overlap})$

Where $A$ is an area.

---

Each term in the formula above is (or can be) encoded in a matrix.

So, it may be possible to write the solution as a series of element-wise and matrix-wise operations (instead of iterating over the elements).

---

- $\kappa_{pixel}$ would be the iterative value 
- $1-A_{overlap}$ would change depending on the current iterator value

==> should be elementwise multiplication

In [20]:
function overlap_matrix(A::GriddedArray, s::Square)
    raster_area(A, s).pixels
end

overlap_matrix (generic function with 1 method)

In [21]:
function overlap_compliment_matrix(A::GriddedArray, s::Square)
     ones(size(A.pixels)) - overlap_matrix(A, s)
end

overlap_compliment_matrix (generic function with 1 method)

# THIS SOLUTION DOES NOT MUTATE THE GRIDDED ARRAY

In [22]:
function raster_scalar_dielectric!(A::GriddedArray, s::Square)
    O = overlap_matrix(A, s)
    C = overlap_compliment_matrix(A, s)
    K_square = s.dielectric .* ones(size(A.pixels))
    
    (A.pixels .* C) + (K_square .* O)
end

raster_scalar_dielectric! (generic function with 1 method)

### Raster Scalar Dielectric Examples

In [23]:
# GriddedArray and Square for a constructed example
A = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(2.0, 5, 5))
s = Square(0.5, 1.0, 0.75, 1.5)

# Apply raster_scalar_dielectric!
raster_scalar_dielectric!(A, s)

5×5 Matrix{Float64}:
 2.0  2.0    2.0  2.0    2.0
 2.0  1.875  1.5  1.875  2.0
 2.0  1.875  1.5  1.875  2.0
 2.0  1.875  1.5  1.875  2.0
 2.0  2.0    2.0  2.0    2.0

# Question 3

$\langle\varepsilon^{-1}\rangle^{-1} = \frac{\kappa_{pixel}\kappa_{square}}{\kappa_{square}(1-A_{overlap}) + \kappa_{pixel}(A_{overlap})}$

# THIS SOLUTION DOES NOT MUTATE THE GRIDDED ARRAY

In [24]:
function raster_harmonic!(A::GriddedArray, s::Square)
    O = overlap_matrix(A, s)
    C = overlap_compliment_matrix(A, s)
    K_square = s.dielectric .* ones(size(A.pixels))
    
    numerator = (A.pixels) .* K_square
    denominator = (K_square .* C) .+ (A.pixels .* O)
    
    numerator ./ denominator
end 

raster_harmonic! (generic function with 1 method)

### Raster Harmonic Example

In [25]:
# GriddedArray and Square for a constructed example
A = GriddedArray((0.0, 1.0), (0.0, 2.0), fill(2.0, 5, 5))
s = Square(0.5, 1.0, 0.75, 1.5)

# Apply raster_scalar_dielectric!
raster_harmonic!(A, s)

5×5 Matrix{Float64}:
 2.0  2.0      2.0  2.0      2.0
 2.0  1.84615  1.5  1.84615  2.0
 2.0  1.84615  1.5  1.84615  2.0
 2.0  1.84615  1.5  1.84615  2.0
 2.0  2.0      2.0  2.0      2.0

# Question 4

Consider a square `s` with 
- center (`s.x`, `s.y`)
- width `s.w`

and point `p` with coordinates (`p_x`, `p_y`)

---

`s` has four corners with coordinates:
- urc = (s.x + s.w/2, s.x + s.w/2)
- ulc = (s.x - s.w/2, s.x + s.w/2)
- llc = (s.x - s.w/2, s.x - s.w/2)
- lrc = (s.x + s.w/2, s.x - s.w/2)

`s` has four surfaces that are open intervals:
- top = (ulc[1], urc[1])
- bot = (llc[1], lrc[1])
- left = (llc[2], ulc[2])
- right = (lrc[2], urc[2])

Each surface has a normal vector:
- $\hat n_{top}$ = [0, 1]
- $\hat n_{bot}$ = [0, -1]
- $\hat n_{left}$ = [-1, 0]
- $\hat n_{top}$ = [1, 0]

---

Say that point `p` is **inside** `s` when one of the following statements is true:
- Point is the Square center:  `p` has the coordinates (s.x, s.y)"
- Point is a corner:  `p` has the coordinates of urc, ulc, llc, or lrc 
- Point is on a surface:  `p_i` ≈ `s.i` and `s_j - s.w/2` < `p_j` < `s_j + s.w/2` (i, j $\leftrightarrow$ x, y)
- Square contains the Point:  `s_i - s.w/2` < `p_i` < `s_i + s.w/2` and `s_j - s.w/2` < `p_j` < `s_j + s.w/2` (i, j $\leftrightarrow$ x, y)

Say that point `p` is outside when `p` is not inside.

---

If one of:
- Point is the Square Center
- Point is a corner
- Point is on a surface

is true then the problem is solved by:
- $\hat 0$
- $\langle$ adjacent surface normals $\rangle$
- $\hat n_{surface}$

respectively

---

If "Square contains Point" or Point is outside, the following procedure should yield the answer.

```
Define s according to 8 coordinate tuples
    - 4 corners
    - 4 midpoints of surfaces
    
    
If p is inside
    For each midpoint
        Construct a vector from p to the midpoint
        Record the value of the perpendicular coordinate
        Record the surface corresponding to this midpoint
Else (p is outside)
    For each midpoint
        Construct a vector from p to the midpoint
        If the perpendicular coordinate of the vector is non-zero
            Record the value of the perpendicular coordinate
            Record the surface correponding to this midpoint

For each corner
    Construct a vector from p to the corner
    Record the length of the vect
    Record the corner
    
If p is outside
    For each midpoint pair
        Calc coordinates of q the recorded distance away from p in the direction of the perpendicular
            If q is not on s
                Discard the pair

If the point is inside
    Calc minimum distance
    Discard all records that are > minimum distance
    If there are ties
        Average all corresponding normals

If the point is outside
    Calc minimum distance 
    Solution is the normal vector of the target with this coordinate
    
```

In [26]:
# Example data
simple_square = Square(0.0, 0.0, 2.0, 1.0)
center = (0, 0)
inside = (0.1, -0.4)
surface = (1.0, 0.5)
corner = (1.0, 1.0)
outside = (-3, 4)

my_points = (center, inside, surface, corner, outside)

((0, 0), (0.1, -0.4), (1.0, 0.5), (1.0, 1.0), (-3, 4))

In [27]:
# Equality comparison for Tuples{Float64, ...}
function fuzzy_eq(a::Tuple, b::Tuple)
    (a[1] ≈ b[1]) && (a[2] ≈ (b[2]))
end

fuzzy_eq (generic function with 1 method)

In [28]:
# Less than or equal to comparison for Float
function fuzzy_leq(left, right)
    (left < right) || (left ≈ right)
end

fuzzy_leq (generic function with 1 method)

In [29]:
# Check if x is inside the open range (lep, rep)
function fuzzy_in_open_range(lep, x, rep)
    
    # Since the range is open, x cannot be lep or rep
    if x ≈ lep
        return false
    end
    
    if x ≈ rep
        return false
    end
       
    # Only eval to true when x is strictly in the open range
    fuzzy_leq(lep, x) && fuzzy_leq(x, rep)
    
end

fuzzy_in_open_range (generic function with 1 method)

In [30]:
# Corners
urc(s::Square) = (s.x + s.w/2, s.y + s.w/2)
ulc(s::Square) = (s.x - s.w/2, s.y + s.w/2)
llc(s::Square) = (s.x - s.w/2, s.y - s.w/2)
lrc(s::Square) = (s.x + s.w/2, s.y - s.w/2)

corners(s::Square) = (urc(s), ulc(s), llc(s), lrc(s))

corners (generic function with 1 method)

In [31]:
function point_is_corner(p::Tuple, s::Square)
    # Assume point is not a corner
    result = false
    
    # Compare point with coordinates of the corners
    for corner in (urc(s), ulc(s), llc(s), lrc(s))
        if fuzzy_eq(p, corner)
            result = true
        end
    end

    result
end

point_is_corner (generic function with 1 method)

In [32]:
function which_corner(p::Tuple, s::Square)
    fwd = Dict(
        "urc" => (s.x + s.w/2, s.x + s.w/2),
        "ulc" => (s.x - s.w/2, s.x + s.w/2),
        "llc" => (s.x - s.w/2, s.x - s.w/2),
        "lrc" => (s.x + s.w/2, s.x - s.w/2)
    )
    
    bkd = Dict(v => k for (k, v) in fwd)
    
    bkd[p]
end

which_corner (generic function with 1 method)

In [33]:
# Surfaces
point_on_top(p::Tuple, s::Square) = (p[2] ≈ (s.y + s.w/2)) && fuzzy_in_open_range(ulc(s)[1], p[1], urc(s)[1])
point_on_bot(p::Tuple, s::Square) = (p[2] ≈ (s.y - s.w/2)) && fuzzy_in_open_range(llc(s)[1], p[1], lrc(s)[1])
point_on_left(p::Tuple, s::Square) = (p[1] ≈ (s.x - s.w/2)) && fuzzy_in_open_range(llc(s)[2], p[2], ulc(s)[2])
point_on_right(p::Tuple, s::Square) = (p[1] ≈ (s.x + s.w/2)) && fuzzy_in_open_range(lrc(s)[2], p[2], urc(s)[2])

point_on_right (generic function with 1 method)

In [34]:
function point_is_on_surface(p::Tuple, s::Square)
    result = false
    
    if point_on_top(p, s)
        result = true
    end
        
    if point_on_bot(p, s)
        result = true
    end
    
    if point_on_left(p, s)
        result = true
    end
    
    if point_on_right(p, s)
        result = true
    end
    
    result 
end

point_is_on_surface (generic function with 1 method)

In [35]:
function which_surface(p::Tuple, s::Square)
    surface = ""
    
    if point_on_top(p, s)
        surface = "top"
    end
        
    if point_on_bot(p, s)
        surface = "bot"
    end
    
    if point_on_left(p, s)
        surface = "left"
    end
    
    if point_on_right(p, s)
        surface = "right"
    end
    
    surface 
end

which_surface (generic function with 1 method)

In [36]:
# Final condition that classifies a point as inside
function square_contains_point(p::Tuple, s::Square)
    fuzzy_in_open_range(ulc(s)[1], p[1], urc(s)[1]) && fuzzy_in_open_range(llc(s)[2], p[2], ulc(s)[2])
end

square_contains_point (generic function with 1 method)

In [37]:
function point_is_inside(p::Tuple, s::Square)
    # Assume point is not inside
    result = false
    
    # Check if p is the Square center
    if fuzzy_eq(p, (s.x, s.y))
        result = true
    end
    
    # Check if p is a corner
    if point_is_corner(p, s)
        result = true
    end
    
    # Check if p lies on a surface
    if point_is_on_surface(p, s)
        result = true
    end
    
    # Check if s contains p
    if square_contains_point(p, s)
        result = true
    end
    
    result 
end

point_is_inside (generic function with 1 method)

In [38]:
distance_formula(a::Tuple, b::Tuple) = sqrt((a[1] - b[1])^2 + (a[2] - b[2])^2)

distance_formula (generic function with 1 method)

In [39]:
# Check if dropping a perpendicular from p intersects with a surface
drop_perp_to_top(p::Tuple, s::Square) = point_on_top((p[1], (s.y + s.w/2)), s)
drop_perp_to_bot(p::Tuple, s::Square) = point_on_bot((p[1], (s.y - s.w/2)), s)
drop_perp_to_left(p::Tuple, s::Square) = point_on_left(((s.x - s.w/2), p[2]), s)
drop_perp_to_right(p::Tuple, s::Square) = point_on_right(((s.x + s.w/2), p[2]), s)

drop_perp_to_right (generic function with 1 method)

In [40]:
function distance_to_surface(p::Tuple, s::Square, surface)
    surfaces = Dict(
        "top" => (p[1], (s.y + s.w/2)),
        "bot" => (p[1], (s.y - s.w/2)),
        "left" => ((s.x - s.w/2), p[2]),
        "right" => ((s.x + s.w/2), p[2])
    )
    
    if surface == "top"
        # A surface is only a candidate solution if dropping a perpendicular from p lands on the surface
        if drop_perp_to_top(p, s)
            return distance_formula(p, surfaces[surface])
        # This entry will be eliminated when distances are filtered 
        else
            return Inf
        end
    end
    
    if surface == "bot"
        # A surface is only a candidate solution if dropping a perpendicular from p lands on the surface
        if drop_perp_to_bot(p, s)
            return distance_formula(p, surfaces[surface])
        # This entry will be eliminated when distances are filtered 
        else
            return Inf
        end
    end
        
    if surface == "left"
        # A surface is only a candidate solution if dropping a perpendicular from p lands on the surface
        if drop_perp_to_left(p, s)
            return distance_formula(p, surfaces[surface])
        # This entry will be eliminated when distances are filtered 
        else
            return Inf
        end
    end
                            
    if surface == "right"
        # A surface is only a candidate solution if dropping a perpendicular from p lands on the surface
        if drop_perp_to_right(p, s)
            return distance_formula(p, surfaces[surface])
        # This entry will be eliminated when distances are filtered 
        else
            return Inf  
        end
    end

end

distance_to_surface (generic function with 1 method)

In [41]:
function distance_to_corner(p::Tuple, s::Square, corner)
    corners = Dict(
        "ulc" => ulc(s),
        "urc" => urc(s),
        "llc" => llc(s),
        "lrc" => lrc(s)
    )
    distance_formula(p, corners[corner])
end

distance_to_corner (generic function with 1 method)

In [42]:
function normalize!(vector)
    magn = distance_formula((0, 0), (vector[1], vector[2]))
    vector = vector ./ magn
    vector
end

normalize! (generic function with 1 method)

In [43]:
function surface_normal(s::Square, coord)
    solution_map = Dict(
    "top" => [0.0 1.0],
    "bot" => [0.0 -1.0],
    "left" => [-1.0 0.0],
    "right" => [1.0 0.0],
    "urc" => [1/sqrt(2) 1/sqrt(2)],
    "ulc" => [-1/sqrt(2) 1/sqrt(2)],
    "llc" => [-1/sqrt(2) -1/sqrt(2)],
    "lrc" => [1/sqrt(2) -1/sqrt(2)],
    "zero" => [0.0 0.0]
)
    
    # When coord is inside, there are some special cases that immediately give the solution
    if point_is_inside(coord, s)
        # Check if coord is the Square center
        if fuzzy_eq(coord, (s.x, s.y))
            return solution_map["zero"]
        end

        # Check if coord is a corner
        if point_is_corner(coord, s)
            return solution_map[which_corner(coord, s)]
        end

        # Check if coord lies on a surface
        if point_is_on_surface(coord, s)
            return solution_map[which_surface(coord, s)]
        end
    end
    
    # None of the special cases occurred so, iterate through the distances to each target
    
    # Container for candidate targets
    candidate = Dict()
    min_dist = Inf
    min_target = ""
    
    for surface in ("top", "bot", "left", "right")
        dist2surf = distance_to_surface(coord, s, surface)
        # If dist_to_surface ≈ min_dist
        if dist2surf ≈ min_dist
            # Record a tie in candidate
            candidate[surface] = dist2surf
        # If dist_to_surface < min_dist
        elseif dist2surf < min_dist
            # There is a new minimum distance and corresponding target
            for key in keys(candidate)
                delete!(candidate, key)
            end
            # Record the new min_dist
            min_dist = dist2surf
            # Record the new min_target
            min_target = surface
            # Record the best candidate
            candidate[min_target] = min_dist
        end
    end
    

    for corner in ("ulc", "urc", "llc", "lrc")
        dist2cor = distance_to_corner(coord, s, corner)
        # If dist_to_corner ≈ min_dist
        if dist2cor ≈ min_dist
            # Record a tie
            candidate[min_target] = min_dist
        # If dist_to_corner < min_dist
        elseif dist2cor < min_dist
            # There is a new minimum distance and corresponding target
            for key in keys(candidate)
                delete!(candidate, key)
            end
            # Record the new min_dist
            min_dist = dist2cor
            # Record the new min_target
            min_target = corner
            # Record the best candidate
            candidate[min_target] = min_dist
        end
    end
    
    # Average the best candidates
    solution = [0 0]
    for key in keys(candidate)
        solution = (solution .+ solution_map[key])
    end
    solution = solution ./ size(collect(keys(candidate)), 1)
    normalize!(solution)
    
end

surface_normal (generic function with 1 method)

# Question 5

> ... specifically the last basis vector

~~Since the global space has 2 basis vectors $\hat e_{x}$ and $\hat e_{y}$ the transition matrix should take us to $\hat e_{x}$ $\hat e_{n}$~~

~~So I think the transition matrix will always look like $U = \begin{bmatrix}1 & n_{1}\\ 0 & n_{2} \end{bmatrix}$~~

I now think that 

> the last basis vector

means the normal "becomes" the new $e_{2}$

In other words, rotate $\theta$ until $\hat n_{Q4}$ $\parallel$ $e_{2}$

In [1]:
function orient_along_normal(A, normal)
    # Calculate the amount to rotate by
    rotation_angle = (π/2) - atan(normal[2], normal[1])
    # Populate the rotation matrix
    R = [cos(rad) -sin(rad); sin(rad) cos(rad)]
    # Apply a rotation on A
    R * A
end

orient_along_normal (generic function with 1 method)